In [2]:
!unzip Image-Cropped.zip

Streaming output truncated to the last 5000 lines.
  inflating: Image-Cropped/train/labels/h_015_jpg.rf.90d4a6f9bee4dcc2b3b88fbf4e317365.txt  
  inflating: Image-Cropped/train/labels/h_015_jpg.rf.abea65161ccb5fcd9cd1d4bce453edfe.txt  
  inflating: Image-Cropped/train/labels/h_015_jpg.rf.d6eef93c194648799c729f9cae1cc3fb.txt  
  inflating: Image-Cropped/train/labels/h_016_jpg.rf.0671c8922cb695d4ce02ce4801d39ed2.txt  
  inflating: Image-Cropped/train/labels/h_016_jpg.rf.11a07cfbc081a37a261759b3950a7701.txt  
  inflating: Image-Cropped/train/labels/h_016_jpg.rf.e344a217955fead5f797c7bf5d579e54.txt  
  inflating: Image-Cropped/train/labels/h_017_jpg.rf.84436c26a4cacf655b3874618a09af0e.txt  
  inflating: Image-Cropped/train/labels/h_017_jpg.rf.a5220d2fe0146533bfd36a6e85dbf5a6.txt  
  inflating: Image-Cropped/train/labels/h_017_jpg.rf.eb3070ecc2598ab0adb27363b276e723.txt  
  inflating: Image-Cropped/train/labels/h_019_jpg.rf.5b13fd55c7e1477d1b37f3462b01f797.txt  
  inflating: Image-Cropped/tr

In [3]:
import cv2
import os
from cv2 import dnn_superres
import matplotlib.pyplot as plt



In [9]:
size = (224, 224)


def data_preprocessing(folder_features, folder_labels, features, labels):
    features_list = os.listdir(folder_features) #files now  contains all the images found in'folder'
    labels_list = os.listdir(folder_labels)
    features_list.sort()
    labels_list.sort()

    for feature, label in zip(features_list, labels_list):
      feature_path = folder_features + '/' + feature
      label_path = folder_labels + '/' + label

      print(f'features: {feature_path}')
      print(f'labels: {label}')

      img = cv2.imread(feature_path)
      img = cv2.resize(img, dsize = size)
      label_file = open(label_path)
      label_class = label_file.readline().split()[0]

      print(f"label class: {label_class}")

      features.append(img)
      labels.append(label_class)

      # name = feature.split('.')[0]
      # img = cv2.imread(folder_features+'/'+feature)
      # label = open(folder_labels+'/'+)
      # name = file.split('.')[0]
      # if img is not None: # if the file is not empty
      #     features.append() # normalization
path_to_train_images = '/content/Image-Cropped/train/images'
path_to_train_labels = '/content/Image-Cropped/train/labels'
path_to_test_images = '/content/Image-Cropped/test/images'
path_to_test_labels = '/content/Image-Cropped/test/labels'
path_to_val_images = '/content/Image-Cropped/valid/images'
path_to_val_labels = '/content/Image-Cropped/valid/labels'

In [10]:
images = []
labels = []

data_preprocessing(path_to_train_images, path_to_train_labels, images, labels)

Streaming output truncated to the last 5000 lines.
labels: s_006_jpg.rf.8c2b2fa24079aa025ff1ccfa9b7f742e.txt
label class: 18
features: /content/Image-Cropped/train/images/s_006_jpg.rf.d6efebee4063acd6a76809b42ba656ec.jpg
labels: s_006_jpg.rf.d6efebee4063acd6a76809b42ba656ec.txt
label class: 18
features: /content/Image-Cropped/train/images/s_007_jpg.rf.01f2d05a311fe8bdf7583b4f1bbe1a2f.jpg
labels: s_007_jpg.rf.01f2d05a311fe8bdf7583b4f1bbe1a2f.txt
label class: 18
features: /content/Image-Cropped/train/images/s_007_jpg.rf.a95575d09d41191e792bdd704edc703e.jpg
labels: s_007_jpg.rf.a95575d09d41191e792bdd704edc703e.txt
label class: 18
features: /content/Image-Cropped/train/images/s_007_jpg.rf.ed4e5fcee1d5c1089299593f6b0fef4d.jpg
labels: s_007_jpg.rf.ed4e5fcee1d5c1089299593f6b0fef4d.txt
label class: 18
features: /content/Image-Cropped/train/images/s_008_jpg.rf.9e825d0dc72103f382465b7b12c8b51d.jpg
labels: s_008_jpg.rf.9e825d0dc72103f382465b7b12c8b51d.txt
label class: 18
features: /content/Image-

In [34]:

'''
- EDSR is very accurate but slow inference time
- FSRCNN is not accurate but quick
- ESPCN is the middle ground with accuracy and very fast inference time (x4)

Downscaled images by 4, rescaled them with standard bicupid interpolation,
and upscaled them with each model
'''
# Create an SR object - only function that differs from c++ code
sr = dnn_superres.DnnSuperResImpl_create()
# Read image
row = 1
for i in range(0, 15, 5):
  image = images[i]
  image = cv2.resize(image, dsize=None, fx=1/4, fy=1/4)

  # Read the desired model
  path = "ESPCN_x4.pb"
  sr.readModel(path)
  # Set the desired model and scale to get correct pre- and post-processing
  sr.setModel("espcn", 4)
  # Upscale the image
  result = sr.upsample(image)
  # Save the image

  resized = cv2.resize(image,dsize=None,fx=4,fy=4) # bicupid interpolation, averages 16 pixels nearby and interpolates rgb values, adds more pixels

  plt.figure(figsize=(200, 200))
  plt.subplot(row,3,1)
  # Reverse BGR to RGB
  plt.imshow(image[:,:,::-1])
  plt.subplot(row,3,2)
  # SR upscaled
  plt.imshow(result[:,:,::-1])
  plt.subplot(row,3,3)
  # OpenCV upscaled
  plt.imshow(resized[:,:,::-1])
  row += 1
plt.show()
plt.ioff()
